## NATURAL LANGUAGE TO SQL 

In this project we will create a simple tool for people to use English text to query a given set of data. 
ChatGPT API call will translate from simple English to SQL which will then query the data and provide the resultset.

Example:

    User Input : Total Sales per month
    GPT generated SQL : SELECT MONTH_ID, SUM(SALES) AS TOTAL_SALES FROM Sales GROUP BY MONTH_ID

In [1]:
#Install OpenAI if not installed already. This should be run only once.
!pip install openai

In [2]:
# Import all necessary packagesß
import openai
import os

In [3]:
#Set the OPENAI API Key to an environment variable. 
#NOTE: Make sure not to share this key on public. Delete this cell after successful run. 
#I have removed key below after the run
OPENAI_API_KEY = 'sk-...'
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
 openai.api_key = os.getenv('OPENAI_API_KEY')


In [5]:
#import pandas to work with dataframes
import pandas as pd

In [6]:
#Download a publically available data for this demo project and load the data csv file to a dataframe
#Data source : https://www.kaggle.com/datasets/kyanyoga/sample-sales-data
df = pd.read_csv('data/sales_data_sample.csv')

In [7]:
#check data
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME
0,10107,30,95.70,2871.00,2/24/2003 0:00,1,2,2003,Motorcycles,2125557818,897 Long Airport Avenue,NYC,NY,10022,USA,Yu,Kwai
1,10121,34,81.35,2765.90,5/7/2003 0:00,2,5,2003,Motorcycles,26.47.1555,59 rue de l'Abbaye,Reims,NaN,51100,France,Henriot,Paul
2,10134,41,94.74,3884.34,7/1/2003 0:00,3,7,2003,Motorcycles,+33 1 46 62 7555,27 rue du Colonel Pierre Avia,Paris,NaN,75508,France,Da Cunha,Daniel
3,10145,45,83.26,3746.70,8/25/2003 0:00,3,8,2003,Motorcycles,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,USA,Young,Julie
4,10159,49,100.00,5205.27,10/10/2003 0:00,4,10,2003,Motorcycles,6505551386,7734 Strong St.,San Francisco,CA,NaN,USA,Brown,Julie


In [8]:
#basic info about the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2823 entries, 0 to 2822
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ORDERNUMBER       2823 non-null   int64  
 1   QUANTITYORDERED   2823 non-null   int64  
 2   PRICEEACH         2823 non-null   float64
 3   SALES             2823 non-null   float64
 4   ORDERDATE         2823 non-null   object 
 5   QTR_ID            2823 non-null   int64  
 6   MONTH_ID          2823 non-null   int64  
 7   YEAR_ID           2823 non-null   int64  
 8   PRODUCTLINE       2823 non-null   object 
 9   PHONE             2823 non-null   object 
 10  ADDRESSLINE1      2823 non-null   object 
 11  CITY              2823 non-null   object 
 12  STATE             1337 non-null   object 
 13  POSTALCODE        2747 non-null   object 
 14  COUNTRY           2823 non-null   object 
 15  CONTACTLASTNAME   2823 non-null   object 
 16  CONTACTFIRSTNAME  2823 non-null   object 


In [9]:
# We can use Pandas to Query this dataframe
#Example - Sum of Sales by Month 
df.groupby("MONTH_ID").sum()['SALES']

MONTH_ID
1      785874.44
2      810441.90
3      754501.39
4      669390.96
5      923972.56
6      454756.78
7      514875.97
8      659310.57
9      584724.27
10    1121215.22
11    2118885.67
12     634679.12
Name: SALES, dtype: float64

### SQL Database

In [10]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [11]:
#Push data into a table called 'Sales'
db = create_engine('sqlite:///:memory:', echo=False)
data = df.to_sql(name='Sales',con=db)

## Connecting to SQL Database:

In [12]:
#crate a connection to the DB and check if it fetches result when queried
with db.connect() as conn:
    result = conn.execute(text("Select ORDERNUMBER, SALES from Sales ORDER BY SALES DESC LIMIT 1"))

In [13]:
result.all()

[(10407, 14082.8)]

###  GPT needs to know the table schema before we can query it.

In [14]:
#The below function provides the table schema (based on the dataframe column names). 
#This will act as first part of the promt we provide to GPT

def create_table_definition_prompt(df):
    prompt = ''' We have a table with below columns:

            Sales({})

            '''.format(",".join(str(x) for x in df.columns))
    
    return prompt

In [15]:
print(create_table_definition_prompt(df))

 We have a table with below columns:

            Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)

            


In [16]:
#The below function asks user for information to be queried.
def prompt_input():
    nlp_text = input("Enter information you want to query from the table : \n ")
    return nlp_text

In [17]:
#User Input : Type in what you want to query from the table
nlp_text = prompt_input()

Enter information you want to query from the table : 
 Total sales by Month


In [18]:
#Combine to ceate the complete prompt string that should be fed to GPT.
#The first word "Select" is provided to enable GPT to start generating the next words in the SQL
def combine_prompts(df, query_prompt):
    definition = create_table_definition_prompt(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition+query_init_string
    
combine_prompts(df, nlp_text)    

' We have a table with below columns:\n\n            Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n\n            ### A query to answer: Total sales by Month\nSELECT'

In [19]:
#Call to OpenAI API to get the generated SQL
response = openai.Completion.create(
  model="text-davinci-003",
  prompt=combine_prompts(df, nlp_text),
  temperature=0,
  max_tokens=150,
  top_p=1.0,
  frequency_penalty=0.0,
  presence_penalty=0.0,
  stop=["#", ";"]
)

In [20]:
response

<OpenAIObject text_completion id=cmpl-7MblecaiRg253oqAZX87i60jsX2cl at 0x7fb44879a090> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " MONTH_ID, SUM(SALES)\nFROM Sales\nGROUP BY MONTH_ID"
    }
  ],
  "created": 1685622750,
  "id": "cmpl-7MblecaiRg253oqAZX87i60jsX2cl",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 22,
    "prompt_tokens": 107,
    "total_tokens": 129
  }
}

In [21]:
#Capture the generated SQL
def handle_response(response):
    query = response["choices"][0]["text"]
    if query.startswith(" "):
        query = "Select"+ query
    return query

In [22]:
handle_response(response)

'Select MONTH_ID, SUM(SALES)\nFROM Sales\nGROUP BY MONTH_ID'

In [23]:
#Run the generated query in the database
# Running within 'with' block will 
# 1. create the connection
# 2.run the code inside the connection and 
# 3.then close the connection
with db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

In [24]:
#save the result
df_result = result.all()

In [25]:
df_result

[(1, 785874.4400000008),
 (2, 810441.9),
 (3, 754501.3900000001),
 (4, 669390.9600000003),
 (5, 923972.56),
 (6, 454756.77999999985),
 (7, 514875.9700000001),
 (8, 659310.5699999998),
 (9, 584724.2699999999),
 (10, 1121215.2199999997),
 (11, 2118885.67),
 (12, 634679.1199999998)]

In [26]:
#save the result in a Pandas dataframe and provide column names for ease of readability
df_result = pd.DataFrame(df_result)
df_result.columns = ['Month_ID', 'Sum of Sales']

In [27]:
#Display results
df_result.set_index('Month_ID')

,Sum of Sales
Month_ID,
1,785874.44
2,810441.90
3,754501.39
4,669390.96
5,923972.56
6,454756.78
7,514875.97
8,659310.57
9,584724.27
